In [1]:
import pyfeats
from typing import *
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from utils import *

In [3]:
llf = LowLevelFeatureExtractor(function=pyfeats.glcm_features, 
                               params={'ignore_zeros': True}, 
                               features_set=['features_mean', 'features_range'])

# root_folder = "C:\\Users\\trong\\Documents\\skin_data"
root_folder = "/mnt/c/Users/trong/Documents/skin_data" 

In [2]:
# Define image transformations
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.Grayscale(num_output_channels=1),    # Convert to grayscale
    ToNumpy(),  # Convert to tensor
])

In [4]:
train_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_train_linux.csv', root_dir=root_folder, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)   

test_dataset = CSVImageMetadataDataset(csv_file='./data/vaynen_test_linux.csv', root_dir=root_folder, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check one batch
images, metadata, labels = next(iter(train_dataloader))
print(images.shape)  # Example: torch.Size([32, 3, 128, 128])
print(metadata.shape)  # Example: torch.Size([32, 6])  -> 6 metadata features
print(labels.shape)  # Example: torch.Size([32])

torch.Size([32, 128, 128])
torch.Size([32, 6])
torch.Size([32])


In [11]:
llf(images)

array([[ 2.05505997e-03,  3.53190530e+01,  8.91263786e-01,
         1.62481748e+02,  2.82395928e-01,  2.65645912e+02,
         6.14607937e+02,  6.59752654e+00,  9.84860911e+00,
         6.28484234e-04,  3.60679566e+00, -2.54945251e-01,
         9.69261168e-01,  3.81110728e+00,  5.73742580e-04,
         2.20666768e+01,  6.83567532e-02,  8.19493057e-01,
         6.65825099e-02,  2.64798655e-02,  2.53446490e+01,
         2.05171835e-02,  4.31453956e-01,  2.14546542e-04,
         4.61914236e-01,  7.74780252e-02,  2.47485734e-02,
         2.57447938e-01],
       [ 4.61721649e-04,  1.26428832e+02,  9.73247786e-01,
         2.36581270e+03,  2.40758392e-01,  2.60014773e+02,
         9.33682196e+03,  8.38284706e+00,  1.18471147e+01,
         3.75732713e-04,  4.04502143e+00, -4.00383815e-01,
         9.98553768e-01,  4.20988472e+00,  2.05841532e-04,
         1.09974904e+02,  2.35539445e-02,  3.03861668e+01,
         9.69746365e-02,  1.01685531e+00,  2.30787996e+02,
         1.86374614e-02,  5.60

In [8]:
import numpy as np

# Simulating a batch tensor (random example)
batch_tensor = np.random.randn(32, 1, 128, 128)  # Shape (32,1,128,128)

# Process each image separately
processed_images = []
for img in batch_tensor:  # Automatically iterates over axis 0
    processed_img = img * 2  # Example operation: scaling by 2
    processed_images.append(processed_img)

# Stack them back to maintain original shape
final_batch_tensor = np.stack(processed_images, axis=0)

print(final_batch_tensor.shape)  # Output: (32, 1, 128, 128)


(32, 1, 128, 128)


In [ ]:
model = SimpleNeuralNetwork(inputs = llf.get_features_size())

# Check the model's architecture
print(model)

In [ ]:
train_model(model, train_dataloader, epochs=100, llf=llf ,features_set=llf.function.__name__, )

In [ ]:
evaluate_model(model, test_dataloader, features_set=llf.function.__name__)

In [5]:
for llf_param in param_list[:]:
    llf = LowLevelFeatureExtractor(**llf_param)
    print(f'{llf.function.__name__} -> {llf(images)}')

fos -> [[ 1.22032904e+02  1.28646233e+03  1.24000000e+02  1.30000000e+02
  -2.17980098e-01  2.50980217e+00  8.00785005e-03  4.97389026e+00
   2.70000000e+01  2.15000000e+02  9.12502147e-02  7.10000000e+01
   9.80000000e+01  1.47000000e+02  1.69000000e+02  9.80000000e+01]
 [ 9.09646446e+01  9.92083754e+02  8.40000000e+01  6.60000000e+01
   5.81860523e-01  2.20903857e+00  1.07910569e-02  4.68494245e+00
   3.80000000e+01  1.86000000e+02  1.00755095e-01  5.60000000e+01
   6.50000000e+01  1.13000000e+02  1.41000000e+02  8.50000000e+01]
 [ 1.45904779e+02  2.06165510e+02  1.46000000e+02  1.45000000e+02
  -7.48343452e-01  6.10461908e+00  2.30636129e-02  4.01289025e+00
   6.80000000e+01  2.08000000e+02  8.28146002e-02  1.30000000e+02
   1.38000000e+02  1.54000000e+02  1.62000000e+02  3.20000000e+01]
 [ 1.37917586e+02  3.51241525e+03  1.56000000e+02  1.68000000e+02
  -7.89032633e-01  2.48525659e+00  6.59879163e-03  5.18972478e+00
   1.10000000e+01  2.40000000e+02  9.05613010e-02  2.70000000e+01


/home/tminh/anaconda3/lib/python3.12/site-packages/pyfeats/multiscale/dwt.py:63: RuntimeWarning: Mean of empty slice.
  features[i][0], features[i][1] = abs(D).mean(), abs(D).std()
/home/tminh/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/tminh/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/tminh/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/tminh/anaconda3/lib/python3.12/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


wp_features -> [[17.67127398 17.50055824 30.88214513 ...  1.96003553  1.45834186
   1.78613593]
 [17.1411279  17.53400134 21.92581368 ...  0.79341538  0.91791633
   1.37232838]
 [11.00736088 10.9550706  21.92532958 ...  1.98621191  1.76341483
   1.64536577]
 ...
 [10.0137968  10.16327296 14.497678   ...  0.39106451  0.37666406
   0.40245869]
 [12.66596737 14.48337444 11.32730651 ...  1.55669328  1.61725073
   1.62327183]
 [13.91084784 15.06967802 19.14999006 ...  1.66920581  1.47632669
   1.4364826 ]]
gt_features -> [[1.03949994 4.41963224 0.41898699 2.44957432 0.23002658 1.8136276
  0.2056396  0.51052175 1.0213677  3.68541454 0.42686621 2.25343947
  0.23762521 1.48273762 0.20568262 0.49712921]
 [0.89005949 3.65046044 0.34899438 2.14557029 0.19083823 1.17384083
  0.15686683 0.30566681 0.83266771 3.18077984 0.32939284 1.88708523
  0.1964266  1.00513455 0.15695355 0.31644126]
 [1.28417675 4.79116557 0.52996495 2.93938163 0.28950652 1.19923817
  0.24648609 0.49971751 1.18878795 3.88589245

In [ ]:
param_list[0]